In [ ]:
import shap
import joblib
import pandas as pd
from imblearn.pipeline import Pipeline
import sys
import os

parent_dir = os.path.abspath("..")  # Remonte d'un niveau
sys.path.append(parent_dir)

from XGBTrainer import XGBTrainer

ModuleNotFoundError: No module named 'XGBTrainer'

In [6]:
pipeline = joblib.load("../models/xgboost_0.6.joblib")
X = pd.read_csv('../data/processed/merged_data_2019_2022.csv')

# 1. Séparer la pipeline en 2 parties : transformations + modèle
preprocessor_and_pca = Pipeline([
    ("preprocessing", pipeline.named_steps["preprocessing"]),
    ("variance", pipeline.named_steps["variance"]),
    ("undersampler", pipeline.named_steps["undersampler"]),  # souvent inactif en prédiction
    ("pca", pipeline.named_steps["pca"]),
])
model = pipeline.named_steps["clf"]

AttributeError: Can't get attribute 'XGBTrainer' on <module '__main__'>

In [ ]:
# 2. Appliquer les transformations
X_transformed = preprocessor_and_pca.transform(X)

# X_transformed est maintenant un numpy array (après PCA), ex. shape = (n, k)
# On peut bricoler des noms de colonnes style "PC1", "PC2", etc. si on veut
import numpy as np
pc_names = [f"PC{i+1}" for i in range(X_transformed.shape[1])]

In [ ]:
# 3. Créer l'explainer, si votre base_estimator est un modèle de type arbres
explainer = shap.TreeExplainer(model)

# 4. Calculer les valeurs SHAP
shap_values = explainer.shap_values(X_transformed)

In [ ]:
# 5. Afficher un summary plot
#   On passe "feature_names" pour que Shap les affiche
shap.summary_plot(shap_values, features=X_transformed, feature_names=pc_names)

In [ ]:
# 6. Waterfall plot pour la classe 1, ex. du 1er individu
shap.force_plot(
    explainer.expected_value[1],  # SHAP pour la classe 1
    shap_values[1][0, :],         # 1 = classe 1, 0 = 1er individu
    pc_names
)
